# training using darts library

TL using the same number of IDs is possible

Reshaping does not seem possible.

when the target dataset has more IDs than the source set, we use the same model to predict multiple ids 

the models are trained on the same amount of IDs as the target test set contains 

residuals of the target test set are not considered 

TODO: Fine tuning and saving outputs


In [1]:
from darts import TimeSeries
from darts.utils.losses import *
from darts.models import *
from darts.metrics.metrics import mse, mae, mape

from utils import data_handling, helpers
import torch
import numpy

four_weeks = 24*7*4

/vol/fob-vol7/nebenf21/reinbene/bene/MA/myenv/lib/python3.10/site-packages/statsforecast/core.py:26: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


# Load datasets

In [2]:
# use electricity dataset
electricity_dict = data_handling.format_electricity()

for key, value in electricity_dict.items():
			electricity_dict[key]= data_handling.df_to_tensor(value)

# normalize train and use matrics for val and test
electricity_dict["4_weeks_train"] = electricity_dict["train"][four_weeks:,:]
electricity_dict["train"], train_standardize_dict = helpers.custom_standardizer(electricity_dict["train"])
electricity_dict["validation"], _ = helpers.custom_standardizer(electricity_dict["validation"], train_standardize_dict)
electricity_dict["test"], _ = helpers.custom_standardizer(electricity_dict["test"], train_standardize_dict)
electricity_dict["4_weeks_train"], _ = helpers.custom_standardizer(electricity_dict["4_weeks_train"], train_standardize_dict)

Length train set: 209 days, 0:00:00
Length validation set: 34 days, 0:00:00
Saving train, validation and test df for faster loading


In [3]:
# bavaria dataset
data_tensor = data_handling.load_bavaria_electricity()
bavaria_dict, standadizer = data_handling.train_test_split_eu_elec(data_tensor, standardize=True)
bavaria_dict["4_weeks_train"] = bavaria_dict["train"][four_weeks:,:]

/vol/fob-vol7/nebenf21/reinbene/bene/MA/myenv/lib/python3.10/site-packages/IPython/core/completerlib.py:120: UserWarning: using rootmodules_cache requires you to install the `pickleshare` library.
  rootmodules_cache = ip.db.get('rootmodules_cache', {})


In [4]:
# building genome project dataset
data_tensor = data_handling.load_genome_project_data()
gp_dict, standadizer = data_handling.train_test_split_eu_elec(data_tensor, standardize=True)
gp_dict["4_weeks_train"] = gp_dict["train"][four_weeks:,:]

In [4]:
# NOT USED FOR EXPERIMENTS
# euro dataset
#data_tensor = data_handling.eu_electricity_to_tensor()
#euro_dict, standadizer = data_handling.train_test_split_eu_elec(data_tensor, standardize=True)

# Define and train models

In [5]:
def train_models(source_data_train, source_data_val, epochs=1):
    """
    -Convert train/val data into Darts TimeSeries format
    -Instantiate different models
    -Fit models to dataset

    Return: fitted models
    """
    ts_train_source = TimeSeries.from_values(source_data_train)
    ts_val_source = TimeSeries.from_values(source_data_val)
   
   
    # model specific parameters
    # nhits/nbeats
    NUM_STACKS = 4
    NUM_BLOCKS = 2
    NUM_LAYERS = 2
    LAYER_WIDTH = 256
    COEFFS_DIM = 5

    # Training settings:
    LR = 0.0005
    BATCH_SIZE = 32
    NUM_EPOCHS = epochs

    # shared parameters
    LAYER_NORM = 'LayerNorm'
    USE_REVIN = True
    LOSS_FN = torch.nn.MSELoss()
    IN_LEN = 96
    OUT_LEN = 96
    DROPOUT = 0.2  # slightly higher than "generic" because we want more robustnes for TL
    VERBOSE = False

    # reproducibility
    np.random.seed(42)
    torch.manual_seed(42)

    nhits_model = NHiTSModel(
    input_chunk_length=IN_LEN,
    output_chunk_length=OUT_LEN,
    activation='ReLU',
    num_stacks=NUM_STACKS, 
    num_blocks=NUM_BLOCKS, 
    num_layers=NUM_LAYERS, 
    layer_widths=LAYER_WIDTH, 
    expansion_coefficient_dim=COEFFS_DIM, 
    trend_polynomial_degree=2, 
    dropout=DROPOUT, 
    loss_fn=LOSS_FN,
    use_reversible_instance_norm=USE_REVIN,
    pl_trainer_kwargs={
            "enable_progress_bar": True,
            # change this one to "gpu" if your notebook does run in a GPU environment:
            "accelerator": "gpu",},
)

    nbeats_model = NBEATSModel(
        input_chunk_length=IN_LEN,
        output_chunk_length=OUT_LEN,
        batch_size=BATCH_SIZE,
        num_stacks=NUM_STACKS,
        num_blocks=NUM_BLOCKS,
        num_layers=NUM_LAYERS,
        layer_widths=LAYER_WIDTH,
        expansion_coefficient_dim=COEFFS_DIM,
        loss_fn=LOSS_FN,
        use_reversible_instance_norm=USE_REVIN,
        activation='ReLU',
        optimizer_kwargs={"lr": LR},
        pl_trainer_kwargs={
            "enable_progress_bar": True,
            # change this one to "gpu" if your notebook does run in a GPU environment:
            "accelerator": "gpu",},
    )


    transformer_model = TransformerModel(
        input_chunk_length=IN_LEN, 
        output_chunk_length=OUT_LEN,
        output_chunk_shift=0, 
        d_model=LAYER_WIDTH, 
        nhead=4, 
        num_encoder_layers=3, 
        num_decoder_layers=3, 
        dim_feedforward=LAYER_WIDTH, 
        dropout=DROPOUT, 
        activation='relu', 
        loss_fn=LOSS_FN,
        norm_type=None, 
        custom_encoder=None, 
        custom_decoder=None,
        use_reversible_instance_norm=USE_REVIN,
    )

    tsmixer_model = TSMixerModel(
        input_chunk_length=IN_LEN, 
        output_chunk_length=OUT_LEN, 
        output_chunk_shift=0,
        hidden_size=LAYER_WIDTH, 
        ff_size=LAYER_WIDTH, 
        num_blocks=NUM_BLOCKS, 
        activation='ReLU', 
        dropout=DROPOUT, 
        loss_fn=LOSS_FN,
        norm_type=LAYER_NORM, 
        normalize_before=False, 
        use_static_covariates=False,
        use_reversible_instance_norm=USE_REVIN,

    )

    nhits_model.fit(        
        ts_train_source,
        val_series=ts_val_source,
        num_loader_workers=4,
        epochs=NUM_EPOCHS,
        max_samples_per_ts=None,
        verbose = VERBOSE,)

    nbeats_model.fit(
        ts_train_source,
        val_series=ts_val_source,
        num_loader_workers=4,
        epochs=NUM_EPOCHS,
        max_samples_per_ts=None,
        verbose = VERBOSE,

    )

    transformer_model.fit(
        ts_train_source,
        val_series=ts_val_source,
        num_loader_workers=4,
        epochs=NUM_EPOCHS,
        max_samples_per_ts=None,
        verbose = VERBOSE,

    )

    tsmixer_model.fit(
        ts_train_source,
        val_series=ts_val_source,
        num_loader_workers=4,
        epochs=NUM_EPOCHS,
        max_samples_per_ts=None,
        verbose = VERBOSE,

    )


    return transformer_model, nbeats_model, tsmixer_model, nhits_model



def evaluate(transformer, nbeats, nhits, tsnmix, target_test):
    """
    Evaluates models on target test set
    

    Input:  -trained models
            -List of target test sets shaped according to models

    Output: [transformer, nhits, tsnmix]-Losses
    """

    forecasting_endpoint = int(len(target_test)) - 96*2
    target_test = TimeSeries.from_values(target_test)
    window = [target_test[i:i+96] for i in range(0, forecasting_endpoint, 5)]
    target = [target_test[i+96:i+96+96] for i in range(0, forecasting_endpoint, 5)]

    # predict over dataloader with slidingwindow implementation and 5 time step shifts for each input
    preds_transformer = transformer.predict(n=96, series=window)
    preds_nhits = nhits.predict(n=96, series=window)
    preds_tsnmix = tsnmix.predict(n=96, series=window)
    preds_nbeats = nbeats.predict(n=96, series=window)


    mse_transformer = mse(preds_transformer, target)
    mse_nhits = mse(preds_nhits, target)
    mse_tsmix = mse(preds_tsnmix, target)
    mse_nbeats = mse(preds_nbeats, target)

    length_loss = len(mse_transformer)

    mae_transformer = mae(preds_transformer, target)
    mae_nhits = mae(preds_nhits, target)
    mae_tsmix = mae(preds_tsnmix, target)
    mae_nbeats = mae(preds_nbeats, target)


    #mape_transformer = mape(preds_transformer, target)
    #mape_nhits = mape(preds_nhits, target)
    #mape_tsmix = mape(preds_tsnmix, target)
    #mape_nbeats = mape(preds_nbeats, target)



    mse_mae_mape_transformer = sum(mse_transformer) / length_loss, sum(mae_transformer) / length_loss#, sum(mape_transformer) / length_loss
    mse_mae_mape_nhits = sum(mse_nhits) / length_loss, sum(mae_nhits) / length_loss#, sum(mape_nhits) / length_loss
    mse_mae_mape_tsmix = sum(mse_tsmix) / length_loss, sum(mae_tsmix) / length_loss#, sum(mape_tsmix) / length_loss
    mse_mae_mape_nbeats = sum(mse_nbeats) / length_loss, sum(mae_nbeats) / length_loss#, sum(mape_nbeats) / length_loss

    return mse_mae_mape_transformer, mse_mae_mape_nhits, mse_mae_mape_nbeats, mse_mae_mape_tsmix

def extend_source_to_target_id_count(source, target):
    source_id_count = source["train"].shape[1]
    target_id_count = target["train"].shape[1]
    full_repeats = target_id_count // source_id_count
    remainder = target_id_count % source_id_count

    repeated_tensor = source["train"].repeat(1, full_repeats)
    remainder_tensor = source["train"][:, :remainder]
    source_train = torch.cat((repeated_tensor, remainder_tensor), dim=1)
    assert target["train"].size(1) == source["train"].size(1), "Reshaping was incorrect."

    repeated_tensor = source["validation"].repeat(1, full_repeats)
    remainder_tensor = source["validation"][:, :remainder]
    source_validation = torch.cat((repeated_tensor, remainder_tensor), dim=1)
    assert target["validation"].size(1) == source["validation"].size(1), "Reshaping was incorrect."

    return source_train, source_validation

In [6]:
def train_and_zero_shot(source_data, target_data):
    # either reshape source or target dataset according to which has less IDs
    source_ids = source_data["train"].size(1)
    target_ids = target_data["test"].size(1)

    fine_tune_horizon = -24*7*4
    target_test = target_data["test"]
    target_fine_tuning = target_data["train"][fine_tune_horizon:,:]

    # remove IDs if source is bigger than target or
    # repeat IDs if target is bigger than source
    if target_ids < source_ids:
        source_train = source_data["train"][:,:target_ids]
        source_validation = source_data["validation"][:,:target_ids]
    else:
        source_train, source_validation = extend_source_to_target_id_count(source_data, target_data)

    transformer, nbeats, tsmix, nhits = train_models(source_train, source_validation, epochs=5)
    return transformer, nbeats, nhits, tsmix, target_test, target_fine_tuning



def fine_tune_models(model, target_fine_tuning, epochs=5):
    """
    Fine tune models over specified epochs

    Input:  -trained models
            -fine tuning dataset
            -epochs

    Returns: fitted models
    """

    ts_fine_tuning = TimeSeries.from_values(target_fine_tuning) 
    NUM_EPOCHS = epochs

    model.fit(
            ts_fine_tuning,
            num_loader_workers=4,
            epochs=NUM_EPOCHS,
            max_samples_per_ts=None,
        )
 
    return model

In [34]:
loss_transformer = {}
loss_nhits = {}
loss_tsnmix = {}
loss_nbeats = {}


transformer, nbeats, nhits, tsmix, target_test, target_fine_tuning = train_and_zero_shot(electricity_dict, bavaria_dict)


Trainer will use only 1 of 3 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=3)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
/vol/fob-vol7/nebenf21/reinbene/bene/MA/myenv/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /vol/fob-vol7/nebenf21/reinbene/bene/MA/myenv/lib/py ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]


`Trainer.fit` stopped: `max_epochs=5` reached.
Trainer will use only 1 of 3 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=3)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]
`Trainer.fit` stopped: `max_epochs=5` reached.
Trainer will use only 1 of 3 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=3)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 H

In [35]:
# electricity as source
loss_transformer["elec_to_bavaria"], loss_nhits["elec_to_bavaria"], loss_tsnmix["elec_to_bavaria"]  = evaluate(transformer, nhits, tsmix, target_test)

transformer_tuned  = fine_tune_models(transformer, target_fine_tuning, epochs=5)
nhits_tuned  = fine_tune_models(nhits, target_fine_tuning, epochs=5)
tsmix_tuned  = fine_tune_models(tsmix, target_fine_tuning, epochs=5)

metrics = evaluate(transformer_tuned, nhits_tuned, tsmix_tuned, target_test)
metrics


Trainer will use only 1 of 3 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=3)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
/vol/fob-vol7/nebenf21/reinbene/bene/MA/myenv/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /vol/fob-vol7/nebenf21/reinbene/bene/MA/myenv/lib/py ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]


Predicting DataLoader 0:  12%|█▏        | 2/17 [00:00<00:00, 28.91it/s]

Predicting DataLoader 0: 100%|██████████| 17/17 [00:00<00:00, 23.50it/s]


Trainer will use only 1 of 3 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=3)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]


Predicting DataLoader 0: 100%|██████████| 9/9 [00:00<00:00, 25.27it/s]


...

Trainer will use only 1 of 3 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=3)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]


Predicting DataLoader 0: 100%|██████████| 9/9 [00:00<00:00, 25.01it/s]


Trainer will use only 1 of 3 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=3)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]


Predicting DataLoader 0: 100%|██████████| 17/17 [00:00<00:00, 38.99it/s]


((0.0008762322704018461, 0.019058848016853022, 0.7485992676493797),
 (0.0008602265230420568, 0.01827123355840079, 0.7163087949156761),
 (1.2296520091706153, 0.9392287497493353, 57.00679485003153))

In [48]:
loss_transformer = {}
loss_nhits = {}
loss_tsnmix = {}

# electricity as source
transformer, nhits, tsmix, target_test, target_fine_tuning = train_and_zero_shot(electricity_dict, bavaria_dict)
loss_transformer["elec_to_bavaria"], loss_nhits["elec_to_bavaria"], loss_tsnmix["elec_to_bavaria"]  = evaluate(transformer, nhits, tsmix, target_test)

transformer, nhits, tsmix, target_test, target_fine_tuning = train_and_zero_shot(electricity_dict, gp_dict)
loss_transformer["elec_to_gp"], loss_nhits["elec_to_gp"], loss_tsnmix["elec_to_gp"] = evaluate(transformer, nhits, tsmix, target_test)


# bavaria as source
transformer, nhits, tsmix, target_test, target_fine_tuning = train_and_zero_shot(bavaria_dict, electricity_dict)
loss_transformer["bavaria_to_elec"], loss_nhits["bavaria_to_elec"], loss_tsnmix["bavaria_to_elec"] = evaluate(transformer, nhits, tsmix, target_test)

transformer, nhits, tsmix, target_test, target_fine_tuning = train_and_zero_shot(bavaria_dict, gp_dict)
loss_transformer["bavaria_to_gp"], loss_nhits["bavaria_to_gp"], loss_tsnmix["bavaria_to_gp"] = evaluate(transformer, nhits, tsmix, target_test)


# genome project as source
transformer, nhits, tsmix, target_test, target_fine_tuning = train_and_zero_shot(gp_dict, bavaria_dict)
loss_transformer["gp_to_bavaria"], loss_nhits["gp_to_bavaria"], loss_tsnmix["gp_to_bavaria"] = evaluate(transformer, nhits, tsmix, target_test)

transformer, nhits, tsmix, target_test, target_fine_tuning = train_and_zero_shot(gp_dict, electricity_dict)
loss_transformer["gp_to_elec"], loss_nhits["gp_to_elec"], loss_tsnmix["gp_to_elec"] = evaluate(transformer, nhits, tsmix, target_test)


Trainer will use only 1 of 3 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=3)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
/vol/fob-vol7/nebenf21/reinbene/bene/MA/myenv/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /vol/fob-vol7/nebenf21/reinbene/bene/MA/myenv/lib/py ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]

  | Name          | Type             | Params
---------------------------------------------------
0 | criterion     | MSELoss          | 0     
1 | train_metr

Epoch 4: 100%|██████████| 76/76 [00:02<00:00, 33.54it/s, train_loss=0.277, val_loss=0.521]

`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: 100%|██████████| 76/76 [00:02<00:00, 33.50it/s, train_loss=0.277, val_loss=0.521]


Trainer will use only 1 of 3 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=3)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]

  | Name                | Type                | Params
------------------------------------------------------------
0 | criterion           | MSELoss             | 0     
1 | train_metrics       | MetricCollection    | 0     
2 | val_metrics         | MetricCollection    | 0     
3 | rin                 | RINorm              | 134   
4 | encoder             | Linear              | 4.4 K 
5 | positional_encoding | _PositionalEncoding | 0     
6 | transformer         | Transformer         | 548 K 
7 | decoder         

...

Predicting DataLoader 0: 100%|██████████| 18/18 [00:00<00:00, 31.85it/s]


Trainer will use only 1 of 3 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=3)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]


Predicting DataLoader 0: 100%|██████████| 9/9 [00:00<00:00, 16.78it/s]


Trainer will use only 1 of 3 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=3)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]


Predicting DataLoader 0: 100%|██████████| 18/18 [00:00<00:00, 33.11it/s]
